In [12]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "./../../segmentanything/Scripts/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

In [13]:
def func(path: str):
    image = cv2.imread(path)
    image = cv2.resize(image, (0,0), fx=0.5, fy=0.5)
    masks = mask_generator.generate(image)
    max_img, max_val, max_idx, max_part = None, 0, None, None
    for idx, ann in enumerate(sorted(masks, key=(lambda x: x['area']), reverse=True)):
        part = np.sum(ann['segmentation'] != 0) / (ann['segmentation'].shape[0] * ann['segmentation'].shape[1])
        if part < 0.005: continue
        mask_img = np.where(ann['segmentation'][:,:,None][:,:,(0,0,0)], image, 0)
        mask_hsv = cv2.cvtColor(mask_img, cv2.COLOR_BGR2HSV)

        val = max(
            (np.sum(cv2.inRange(mask_hsv, np.array([105, 161, 108]), np.array([112, 255, 210])) != 0) / np.sum(ann['segmentation'] != 0)),
            (np.sum(cv2.inRange(mask_hsv, np.array([38, 30, 133]), np.array([100, 164, 218])) != 0) / np.sum(ann['segmentation'] != 0))
        )
        if val > 0.7: continue
        if val > max_val:
            max_val = val
            max_img = mask_img
            max_idx = idx
            max_part = part
    print(max_val, max_idx, max_part)
    plt.imshow(max_img)